# Simple CNN Model (Single)

## Imports

Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow import set_random_seed

from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from src.sampler import train_test_bootstrapper
from src.model_api import getSimpleModel
from src.prediction import array_filter, array_to_string

Using TensorFlow backend.


Import data

In [3]:
with open('data/derived/data_train.pickle', 'rb') as file:
    data_train = pickle.load(file)
with open('data/derived/labels_train.pickle', 'rb') as file:
    labels_train = pickle.load(file)
print('Shape of data_train:', data_train.shape)
print('Length of labels_train:', len(labels_train))

Shape of data_train: (25361, 224, 224, 3)
Length of labels_train: 25361


In [2]:
with open('data/derived/data_test.pickle', 'rb') as file:
    data_test = pickle.load(file)
with open('data/derived/test_file_names.pickle', 'rb') as file:
    filenames_test = pickle.load(file)
print('Shape of data_test:', data_test.shape)
print('Length of filename_test:', len(filenames_test))

Shape of data_test: (7960, 224, 224, 3)
Length of filename_test: 7960


Convert labels to integers for model training

In [4]:
label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
print('Original labels: {}'.format(labels_train[:5]))
print('Encoded labels: {}'.format(labels_train_encoded[:5]))
print('Label encoder classes: {}'.format(label_encoder.classes_[:5]))

Original labels: ['w_025911c', 'new_whale', 'new_whale', 'new_whale', 'new_whale']
Encoded labels: [44  0  0  0  0]
Label encoder classes: ['new_whale' 'w_0003639' 'w_0003c59' 'w_0027efa' 'w_00289b1']


Compute sample size and number of classes

In [5]:
sample_size = len(labels_train_encoded)
num_classes = len(set(labels_train_encoded))
print('Sample size:', sample_size)
print('Number of clases:', num_classes)

Sample size: 25361
Number of clases: 5005


## Train Single CNN Model

Simple CNN model (Random Seed = 2019, assuming class imbalance)

In [5]:
# Normalize data_train
data_train = data_train / 255
data_train = data_train.astype(dtype=np.float32)

In [6]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getSimpleModel(num_classes=len(set(labels_train_encoded)), resize_width=224)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=True, random_state=2019)

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=2019)
    
np.random.seed(2019)
set_random_seed(2019)
train_size, epochs = X_train.shape[0], 10
model.fit_generator(image_generator_samples, steps_per_epoch=train_size//epochs, 
                    validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                    callbacks=[checkpoint, early_stop])

model.save_weights('models/weights_CNN_single.hdf5')

Epoch 1/10
2536/2536 [==============================] - 756s 298ms/step - loss: 7.1958 - acc: 0.1835 - val_loss: 4.6493 - val_acc: 0.5763
Epoch 2/10
2536/2536 [==============================] - 751s 296ms/step - loss: 6.9585 - acc: 0.1848 - val_loss: 4.8778 - val_acc: 0.5763
Epoch 3/10
2536/2536 [==============================] - 752s 297ms/step - loss: 6.5814 - acc: 0.1834 - val_loss: 4.9661 - val_acc: 0.5762


Load model weights

In [6]:
model = getSimpleModel(num_classes=len(set(labels_train_encoded)), resize_width=224)
model.load_weights('models/weights_CNN_single.hdf5')

Model prediction

data_test = data_test / 255
data_test = data_test.astype(dtype=np.float32)
preds = model.predict(data_test)

Obtain labels with top 5 softmax values for each array row and concatenate labels

In [8]:
top5_indices = np.apply_along_axis(array_filter, arr=preds, axis=1, n_top=5, labels=label_encoder.classes_)
predictions_array = np.apply_along_axis(array_to_string, arr=top5_indices, axis=1)

Create submission DataFrame and export as CSV file

In [9]:
submission_df = pd.DataFrame({'Image': filenames_test, 'Id': predictions_array})
submission_df.to_csv('submission_v3.csv', index=False)
print(submission_df.head())

           Image                                                 Id
0  c303faac6.jpg  new_whale w_eee296c w_1834b49 w_e13f4e8 w_9d29561
1  96c2b7290.jpg  new_whale w_60ce6fc w_6822dbc w_c6b4d61 w_af367c3
2  69f6cd44f.jpg  new_whale w_d405854 w_2b069ba w_04003e9 w_1ca9ab1
3  a965dea33.jpg  new_whale w_8da30ad w_343f088 w_a9304b9 w_8ea9f68
4  9a225e056.jpg  new_whale w_789c969 w_343f088 w_99e43ae w_fdcaaa8


Kaggle score: 0.279